<a href="https://colab.research.google.com/github/Maxxx-VS/CV-Arduino/blob/main/Yolo_mobil_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python opencv-python-headless tflite-runtime numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.7 MB/s eta 0:00:00


In [4]:
# Импорт библиотек
import cv2
import time
import numpy as np
import urllib.request
import tflite_runtime.interpreter as tflite
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# Скачивание модели YOLOv4-tiny в формате Darknet
def download_file(url, filename):
    print(f"Downloading {filename} from {url}...")
    urllib.request.urlretrieve(url, filename)
    print(f"{filename} saved.")

# Скачиваем веса и конфигурацию модели
weights_url = "https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights"
cfg_url = "https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-tiny.cfg"
download_file(weights_url, "yolov4-tiny.weights")
download_file(cfg_url, "yolov4-tiny.cfg")

# Установка инструмента для конвертации Darknet в TFLite
!git clone https://github.com/hunglc007/tensorflow-yolov4-tflite.git
%cd tensorflow-yolov4-tflite
!pip install -r requirements.txt

# Конвертация модели в TensorFlow Lite
!python save_model.py --weights ../yolov4-tiny.weights --output ./checkpoints/yolov4-tiny --input_size 416 --model yolov4 --framework tflite

# Конвертация SavedModel в TFLite
import tensorflow as tf

saved_model_dir = './checkpoints/yolov4-tiny'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Сохранение TFLite модели
with open('yolov4-tiny.tflite', 'wb') as f:
    f.write(tflite_model)

# Загрузка модели YOLOv4-tiny
interpreter = tflite.Interpreter(model_path='yolov4-tiny.tflite')
interpreter.allocate_tensors()

# Получение информации о входных и выходных тензорах
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Параметры модели
input_shape = input_details[0]['shape']
input_width, input_height = input_shape[1], input_shape[2]

# Функция для захвата видеопотока с камеры в Colab
def capture_camera():
    js = Javascript('''
        async function capture() {
            const video = document.createElement('video');
            video.style.display = 'none';
            document.body.appendChild(video);
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            document.body.removeChild(video);
            return canvas.toDataURL('image/jpeg', 0.8);
        }
    ''')
    display(js)
    data = eval_js('capture()')
    binary = b64decode(data.split(',')[1])
    return binary

# Переменные для расчета FPS
frame_count = 0
start_time = time.time()

while True:
    # Захват кадра с камеры
    frame_data = capture_camera()
    frame = cv2.imdecode(np.frombuffer(frame_data, dtype=np.uint8), cv2.IMREAD_COLOR)

    # Преобразование кадра для модели
    resized_frame = cv2.resize(frame, (input_width, input_height))
    input_data = np.expand_dims(resized_frame, axis=0)
    input_data = (input_data / 255.0).astype(np.float32)

    # Запуск инференса
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    # Получение результатов
    boxes = interpreter.get_tensor(output_details[0]['index'])
    scores = interpreter.get_tensor(output_details[1]['index'])
    classes = interpreter.get_tensor(output_details[2]['index'])

    # Отображение результатов в консоль
    for i in range(len(scores[0])):
        if scores[0][i] > 0.5:  # Фильтр по confidence score
            class_id = int(classes[0][i])
            confidence = scores[0][i]
            box = boxes[0][i]
            print(f"Class: {class_id}, Confidence: {confidence:.2f}, Box: {box}")

    # Расчет и вывод FPS
    frame_count += 1
    if frame_count % 10 == 0:
        end_time = time.time()
        fps = frame_count / (end_time - start_time)
        print(f"FPS: {fps:.2f}")
        frame_count = 0
        start_time = time.time()

    # Отображение кадра с bounding boxes (опционально)
    cv2.imshow('YOLOv4-tiny Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cv2.destroyAllWindows()

yolov4-tiny.weights saved.
yolov4-tiny.cfg saved.
Cloning into 'tensorflow-yolov4-tflite'...
remote: Enumerating objects: 790, done.
remote: Total 790 (delta 0), reused 0 (delta 0), pack-reused 790 (from 1)
Receiving objects: 100% (790/790), 197.64 MiB | 17.60 MiB/s, done.
Resolving deltas: 100% (374/374), done.
/content/tensorflow-yolov4-tflite
ERROR: Ignored the following yanked versions: 3.4.11.39, 3.4.17.61, 4.4.0.42, 4.4.0.44, 4.5.4.58, 4.5.5.62, 4.7.0.68
ERROR: Could not find a version that satisfies the requirement opencv-python==4.1.1.26 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.60, 4.5.5.64, 4.6.0.66, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80, 4.10.0.82, 4.10.0.84, 4.11.0.86)
ERROR: No matching distribution found for opencv-python==4.1.1.26
2025-03-10 16:01:48.302684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.

OSError: SavedModel file does not exist at: ./checkpoints/yolov4-tiny/{saved_model.pbtxt|saved_model.pb}